# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [32]:
import numpy as np
import xarray as xr
import xskillscore as xs

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [33]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim=None, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`. For correlation metrics `dim` cannot be `None`.

In [68]:
r = xs.pearson_r(obs, fct, dim='time')
print(r)

<xarray.DataArray (lat: 4, lon: 5)>
array([[ 0.79186726,  0.02162092, -0.26761106, -0.97050347,  0.09376211],
       [ 0.02557609, -0.8553122 , -0.19878814, -0.61246567, -0.91747827],
       [-0.09699106,  0.9899133 , -0.26796342,  0.76749171, -0.57642707],
       [-0.12434802, -0.81618549,  0.87897933, -0.54363725, -0.90799949]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [69]:
p = xs.pearson_r_p_value(obs, fct, dim="time")
print(p)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.4182183 , 0.98623462, 0.82753155, 0.15500794, 0.94022138],
       [0.98371598, 0.34673064, 0.87259887, 0.58035554, 0.26044264],
       [0.93815635, 0.09049724, 0.82729873, 0.44300913, 0.60889297],
       [0.92063215, 0.39216945, 0.31645017, 0.63409348, 0.27521841]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [70]:
r = xs.pearson_r(obs, fct, dim=["lat", "lon"])
print(r)

<xarray.DataArray (time: 3)>
array([-0.50184345, -0.13946489, -0.02688233])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [38]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [39]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [40]:
r_weighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([-0.00044552,  0.00193725,  0.0058369 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [71]:
r_unweighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([0.00202843, 0.00101358, 0.00720416])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [42]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.59882427, 0.34188626, 0.00298211, 0.10186572, 0.48424417],
        [0.3983712 , 0.63105448, 0.4481283 , 0.95842039, 0.56213108]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.50314444, 0.63645111, 0.08979512, 0.45448371, 0.14772549],
        [0.68441339, 0.33051116, 0.33080909, 0.05871267, 0.30809435]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.42372424, 0.41801783, 0.14394869, 0.20230907, 0.05969924],
        [0.57602916, 0.30609991, 0.52542387, 0.31402834, 0.12028987]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) int64 

In [73]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, dim=['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray (time: 3)>
array([0.40732982, 0.2783628 , 0.36063111])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [72]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, dim=['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Brier Score (`brier_score`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [45]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"]
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
   )

Continuous Ranked Probability Score with the ensemble distribution. Pass `dim=[]` to get the same behaviour as `properscoring.crps_ensemble` without any averaging over `dim`.

In [74]:
crps_ensemble = xs.crps_ensemble(obs3, fct3, dim=[])
print(crps_ensemble)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.39511653, 0.10429151, 0.10012419, 0.27029191, 0.60344486],
       [0.23732312, 0.17457164, 0.30750124, 0.13342076, 0.10600197],
       [0.0186343 , 0.18125707, 0.12432469, 0.18311088, 0.20098147],
       [0.07748468, 0.06068627, 0.28721328, 0.31895966, 0.23381981]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [75]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"), dim=[])
print(crps_gaussian)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.38408416, 0.08882234, 0.1073042 , 0.26558913, 0.59640724],
       [0.23420534, 0.17923419, 0.29422063, 0.12502213, 0.06963974],
       [0.01265487, 0.1003267 , 0.12307614, 0.14001281, 0.21651179],
       [0.03716186, 0.07571929, 0.2896284 , 0.31816021, 0.23481026]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [76]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm, dim=[])
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.57339363, 0.35406721, 0.24561143, 0.43449299, 0.52399342],
       [0.25623238, 0.51506961, 0.30411694, 0.39949527, 0.3952749 ],
       [0.2366792 , 0.29013566, 0.58545773, 0.27593837, 0.23385049],
       [0.39626912, 0.23923865, 0.23658637, 0.23406189, 0.26575328]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold.

In [77]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.7, dim=[])
print(threshold_brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.44444444, 0.        , 0.11111111, 1.        , 1.        ],
       [0.        , 0.11111111, 0.44444444, 0.11111111, 0.11111111],
       [0.        , 0.11111111, 0.11111111, 0.11111111, 0.        ],
       [0.11111111, 0.        , 0.11111111, 0.        , 0.44444444]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [78]:
dichotomous_category_edges = np.array([0, 0.5, 1]) # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(obs, fct,
                                        dichotomous_category_edges,
                                        dichotomous_category_edges,
                                        dim=['lat','lon'])
dichotomous_contingency_table = dichotomous_contingency.table

In [90]:
dichotomous_contingency_table

<xarray.DataArray 'histogram_observations_forecasts' (time: 3, observations_category: 2, forecasts_category: 2)>
array([[[5, 5],
        [7, 3]],

       [[5, 7],
        [6, 2]],

       [[9, 6],
        [4, 1]]])
Coordinates:
  * time                          (time) object 2000-01-01 00:00:00 ... 2000-...
    observations_category_bounds  (observations_category) <U10 '(0.0, 0.5]' '...
    forecasts_category_bounds     (forecasts_category) <U10 '(0.0, 0.5]' '(0....
  * observations_category         (observations_category) int64 1 2
  * forecasts_category            (forecasts_category) int64 1 2

In [94]:
dichotomous_contingency_table.to_dataframe().pivot_table(index=['forecasts_category','forecasts_category_bounds'],
                                                         columns=['observations_category','observations_category_bounds']).round(2)

histogram_observations_forecasts  \
observations_category                                                       1   
observations_category_bounds                                       (0.0, 0.5]   
forecasts_category forecasts_category_bounds                                    
1                  (0.0, 0.5]                                            6.33   
2                  (0.5, 1.0]                                            6.00   

                                                         
observations_category                                 2  
observations_category_bounds                 (0.5, 1.0]  
forecasts_category forecasts_category_bounds             
1                  (0.0, 0.5]                      5.67  
2                  (0.5, 1.0]                      2.00

Scores based on the constructed contingency table can be called via class methods. The available methods are:

* `bias_score`
* `hit_rate`
* `false_alarm_ratio`
* `false_alarm_rate`
* `success_ratio`
* `threat_score`
* `equit_threat_score`
* `odds_ratio`
* `odds_ratio_skill_score`
* `accuracy`
* `heidke_score`
* `peirce_score`
* `gerrity_score`

Below, we share a few examples of these in action.

In [95]:
dichotomous_contingency.bias_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.8  , 1.125, 1.4  ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [96]:
dichotomous_contingency.hit_rate()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.3 , 0.25, 0.2 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [97]:
dichotomous_contingency.false_alarm_rate()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.5       , 0.58333333, 0.4       ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [98]:
dichotomous_contingency.odds_ratio_skill_score()


<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.4       , -0.61538462, -0.45454545])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

Now we can leverage multi-category edges to make use of some scores.

In [99]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(obs, fct,
                                              multi_category_edges,
                                              multi_category_edges,
                                              dim=['lat','lon'])

In [100]:
multicategory_contingency.accuracy()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.15, 0.35, 0.4 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [101]:
multicategory_contingency.heidke_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.328125  ,  0.01140684,  0.0438247 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [102]:
multicategory_contingency.peirce_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.32061069,  0.01351351,  0.05045872])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [103]:
multicategory_contingency.gerrity_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.28717949, -0.14313725, -0.10127592])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [64]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [66]:
ds.xs.pearson_r("obs_var", "fct_var", dim="time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[ 0.79186726,  0.02162092, -0.26761106, -0.97050347,  0.09376211],
       [ 0.02557609, -0.8553122 , -0.19878814, -0.61246567, -0.91747827],
       [-0.09699106,  0.9899133 , -0.26796342,  0.76749171, -0.57642707],
       [-0.12434802, -0.81618549,  0.87897933, -0.54363725, -0.90799949]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4

You can also pass in a separate Dataset that contains your observations or forecast variable.

In [67]:
ds = ds.drop_vars("fct_var")
ds.xs.pearson_r("obs_var", fct, dim="time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[ 0.79186726,  0.02162092, -0.26761106, -0.97050347,  0.09376211],
       [ 0.02557609, -0.8553122 , -0.19878814, -0.61246567, -0.91747827],
       [-0.09699106,  0.9899133 , -0.26796342,  0.76749171, -0.57642707],
       [-0.12434802, -0.81618549,  0.87897933, -0.54363725, -0.90799949]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4